In [1]:
import tensorflow as tf
from Exp_gain import *
import json
import yaml

In [2]:
def init_vec(dimension):
    return tf.Variable(tf.random_normal(dimension, stddev=0.01))

In [3]:
class DDRank_1(object):
    def __init__(self, time, n_feature, n_hidden, lr):

        self.time = time
        self.group_size = 5
        self.lr = lr
        self.top_n=10

        global learning_rate, query_docs, candidate_docs, score, policy, train_step, sess

        with tf.name_scope('input'):
            learning_rate = tf.placeholder(tf.float32)
            query_docs = tf.placeholder(tf.float32, [None, n_feature], name='docs')
            candidate_docs = tf.placeholder(tf.float32, [None, n_feature], name='candidate_docs')

        cell = tf.contrib.rnn.GRUCell(n_hidden)

        with tf.name_scope('rnn'):
            ep_split = tf.split(query_docs, 1, 0, 'split')
            _, state = tf.contrib.rnn.static_rnn(cell, ep_split, dtype=tf.float32)

        with tf.name_scope('policy'):
            w = init_vec([n_feature, n_hidden])
            score = tf.tanh(tf.matmul(candidate_docs, tf.matmul(w, tf.reshape(state[-1], [n_hidden, 1]))))
            policy = tf.nn.softmax(tf.transpose(score))

        init = tf.global_variables_initializer()
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=policy, labels=[0])

        train_step = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

        sess = tf.Session()
        sess.run(init)

    def gen_episode_softmax(self, query_vec, doc_vec, docs, truth):
        doc_id = docs[:]

        rank_list = []
        relevance_feedback = []
        for ite in range(self.time):
            relevance_feedback.append([])
            prob = sess.run(policy,feed_dict={query_docs: np.asanyarray(query_vec), candidate_docs: np.asanyarray(doc_vec)})
            
            action_list=[]
            while len(action_list)<self.group_size:
                action = np.random.choice(len(prob[0]), 1, p=prob[0])[0]
                if action in action_list:
                    continue
                action_list.append(action)
                
                rank_list.append(doc_id[action])
                query_vec.append(doc_vec[action])
                if doc_id[action] in truth[0].keys():
                    relevance_feedback[ite].append(doc_vec[action])

            index = sorted(action_list)
            for i in range(len(index)):
                del doc_id[index[i]-i]
                del doc_vec[index[i]-i]
                
        return rank_list, query_vec + doc_vec, relevance_feedback

    def gen_episode_greedy(self, query_vec, doc_vec, docs, truth):
        doc_id = docs[:]

        rank_list = []
        for ite in range(self.time):
            scores = sess.run(score,feed_dict={query_docs: np.asanyarray(query_vec), candidate_docs: np.asanyarray(doc_vec)})

            for i in range(self.group_size):
                action = np.argmax(scores)
                
                scores = np.delete(scores,action)
                rank_list.append(doc_id[action])
                if doc_id[action] in truth[0].keys():
                    query_vec.append(doc_vec[action])
                del doc_id[action]
                del doc_vec[action]
                
        return rank_list

    def test_model(self, query_docs, doc2vec, truth):
        mean_alphaNDCG=np.zeros(self.top_n)
        n_query = len(query_docs.keys())

        for query in query_docs.keys():
            docs = query_docs[query][:]

            n_doc = len(docs)
            if n_doc < 2:
                print '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!', query
                continue

            query_docs_vec = []
            query_docs_vec.append(doc2vec[query])
            docs_vec = []
            for doc in docs:
                docs_vec.append(doc2vec[doc])

            # gen episode
            rank_list= self.gen_episode_greedy(query_docs_vec, docs_vec, docs, truth[query])

            the_alphaNDCG = alphaNDCG_per_query(rank_list, truth[query])
            mean_alphaNDCG += np.asarray(the_alphaNDCG)
            # print reward[0],
        mean_alphaNDCG = mean_alphaNDCG / n_query
        print '\n\t\t', mean_alphaNDCG.tolist()

    def train_model(self, querydocs, doc2vec, truth):
        querylist = querydocs.keys()
        train_data = {}

        mean_reward = 0
        nquery = len(querylist)
        for query in querylist:
            docs = querydocs[query][:]
            
            querydocsvec = []
            querydocsvec.append(doc2vec[query])
            docsvec = []
            for doc in docs:
                docsvec.append(doc2vec[doc])
                
            # print 'gen_episode_softmax'
            ranklist, querydocvec, feedback = self.gen_episode_softmax(querydocsvec, docsvec, docs, truth[query])
            # print ranklist
            reward = alphaDCG_group_reward_per_query(ranklist, truth[query])
            mean_reward += reward[0]
            # print 'reward', reward[0]
            train_data[query] = {'querydocvec': querydocvec, 'reward': reward, 'feedback': feedback}
        mean_reward = mean_reward/nquery
        print mean_reward,

        for query in querylist:
            querydocvec = train_data[query]['querydocvec']
            reward = train_data[query]['reward']
            self.optimize_model(querydocvec, feedback, reward)

    def optimize_model(self, querydocvec, relevance_feedback, reward):
        ndoc = len(querydocvec)
        input = [querydocvec[0]]
        for time in range(len(reward) ):
            if reward[time] == 0:
                continue
            for i in range(self.group_size):
                sess.run(train_step, feed_dict={query_docs: np.asanyarray(input), candidate_docs: np.asanyarray(querydocvec[time + 1:ndoc + 1]), learning_rate: self.lr * reward[i]})
                
            input += relevance_feedback[time]

    def main(self, nite, querydocs, doc2vec, query_document_topic):
        for ite in range(nite):
            self.train_model(querydocs, doc2vec, query_document_topic)

            if ite % 10 == 0:
                self.test_model(querydocs, doc2vec, query_document_topic)

In [4]:
Truth = json.load(open('/home/zw/DD/data/Truth.json'))
print 'load 1'
IdQuery = json.load(open('/home/zw/DD/data/IdQuery.json'))
print 'load 2'
QueryDocs = json.load(open('/home/zw/DD/data/QueryDocs.json'))
print 'load 3'
Doc2Vec = yaml.load(open('/home/zw/DD/data/Doc2Vec.yml'))
print 'load 4'

load 1
load 2
load 3


load 4


In [5]:
Nfeature=200
Nhidden = 10
Lenepisode = 10
lr=0.1

model = DDRank_1(Lenepisode, Nfeature, Nhidden, lr)


model.main(10000, QueryDocs, Doc2Vec, Truth)

0.687123581737

 
		[0.07407407407407407, 0.09371390651318177, 0.09665068215679135, 0.10663506358808424, 0.10460555775107067, 0.1100271787304158, 0.11702712628846866, 0.12364307493149312, 0.12865677545462376, 0.13616952818716804]


0.814828514753

 0.759863404379

 0.790205922416

...

0.767164870156

 0.695233007504

 0.641114940738

 0.634926557856

 0.647429994028